In [19]:
#!jupyter nbconvert --to=python plot_Taylor_normalized.ipynb

[NbConvertApp] Converting notebook plot_Taylor_normalized.ipynb to python
[NbConvertApp] Writing 2548 bytes to plot_Taylor_normalized.py


In [10]:
def plt_Taylor_norm(data_ref,data_pred,
                mask=True,
                title='Normalized Taylor Diagram',
                label=['Observation', 'Model'],
                loc='right'):
    '''
    Use this function to get Taylor statistics and plot them as normalized to std(reference) = 1
    
    Applied normalization: 
        std_normalized(ref) = std(ref) / std(ref)
        std_normalized(pred) = std(pred) / std(ref)
        
    Input:
    data_ref: array with references/observation data
    data_pred: array with model data
    mask: bolean if observation data should be masked with np.where(data_pred !=0)
    title: string for plot title
    label: list with 2 entries for label for 'Observation', 'Model'
    loc: string, positional argument for legend
    '''
    import numpy as np
    import matplotlib.pyplot as plt
    import skill_metrics as sm
    
    if mask == True:
        # get statistics only from ocean gridpoints (same mask assumption as above)
        ind_stat = np.where(data_pred != 0)
        taylor_stats1 = sm.taylor_statistics(data_ref[ind_stat],data_pred[ind_stat])
    else:
        taylor_stats1 = sm.taylor_statistics(data_ref,data_pred)

        
    sdev = np.array([taylor_stats1['sdev'][0]/taylor_stats1['sdev'][0], 
                     taylor_stats1['sdev'][1]/taylor_stats1['sdev'][0]])
    crmsd = np.array([taylor_stats1['crmsd'][0], taylor_stats1['crmsd'][1]])
    ccoef = np.array([taylor_stats1['ccoef'][0], taylor_stats1['ccoef'][1]])


    fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
    sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                          titleOBS = 'observation', markerLabel = label,
                          markerLabelColor = 'c',
                          markerColor = 'c', markerLegend = 'on',
                          #tickRMS = range(0,5,1),
                          tickRMSangle = 135.0,
                          colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                          titleRMS = 'off', #tickSTD = np.arange(0,1.25,.25),
                          #axismax = 1.25, 
                          colSTD = 'b', styleSTD = '-.',
                          widthSTD = 1.0, titleSTD = 'on',
                          colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                          titleCOR = 'on')
    
    plt.title(title, loc=loc)   
